# Part 1 : Index Construction

## Data Crawling

In [3]:
from googlesearch import search 
import tldextract  
import requests 
from bs4 import BeautifulSoup 
import csv   
from time import sleep 

def format_data(doc_id, query, website, content):
    return {
        "doc_id" : doc_id,
        "query" : query,
        "website" : website,
        "content" : content,
    }

# to search 
queries = ["Computer Science", "IITs in India", "Cities of Chattisgarh"]

docs = []

#Loop for each query
for i in range(len(queries)):
    query = queries[i]
    # Loop for each search result
    count_files = 20
    urls = []
    for url in search(query,num=25, start= 1, stop=50, pause=2): 
        urls.append(url)
        
    for url in urls:
        print(url)
        
        # pdf url are not useful for us as we can not extract data 
        #from it and sometimes they can create problem for us that's why we
        #are removing them
        if url[-3:] == 'pdf':
            continue
        
        try:
            respons = requests.get(url, timeout=5) #Response for the current URL
        except:
            continue
        soup = BeautifulSoup(respons.content, 'html5lib')
        paras=soup.find_all('p')  #Finding all the paragraph within the response
        para = "" #Initializing a empty paragraph
        
        count = 2 #To insure that file must contains 2 paragraph
        flag = 0
        
        for p in paras:   # Loop though each paragraph in response and append the contnet of that paragraph to  "para" so that we can save it as a continue paragraph
            if p!="":
                para += p.get_text()
                count-=1
            if (count==0):
                flag=1
                break
                
        if(flag):  #Perfom writing to file onlw when there is something in para
            ext_url = tldextract.extract(url)  #parsing the URL 
            domin_name = ext_url.domain   #Get the domain name from the parsed URL so that we can use it as a fine name
            
            docs.append(format_data(doc_id = f'd{21-count_files}_q{i+1}',
                                            query = query,
                                            website=url,
                                            content=para
                                           )
                               )
            count_files-=1
        
        if count_files==0:
            break

https://www.britannica.com/science/computer-science
https://www.edx.org/course/subject/computer-science
https://www.khanacademy.org/computing/computer-science
https://www.topuniversities.com/courses/computer-science-information-systems/guide
https://www.internationalstudent.com/study-computer-science/what-is-computer-science/
https://www.coursera.org/browse/computer-science
https://www.youtube.com/watch?v=ReVeUvwTGdU
https://www.seas.harvard.edu/computer-science
https://engineering.virginia.edu/departments/computer-science
https://engineering.buffalo.edu/computer-science-engineering.html
https://www.computerscience.org/
https://www.usnews.com/best-graduate-schools/top-science-schools/computer-science-rankings
https://www.timeshighereducation.com/student/what-to-study/computer-science
https://engineering.tamu.edu/cse/index.html
https://cse.umn.edu/cs
https://arxiv.org/archive/cs
https://www.soe.ucsc.edu/departments/computer-science-and-engineering
https://github.com/ossu/computer-scienc

## Inverted Index

Associate a collection of terms (lexicon) with the documents that contain those terms.
The data structure is much more dense than a Document Term Matrix.

In [3]:
# Collection of documents (corpus)

'''
import json

with open('docs.json') as json_file:
    docs = json.load(json_file)
'''
import json

with open('docs.json') as json_file:
    docs = json.load(json_file)
docs 

[{'doc_id': 'd1_q1',
  'query': 'Computer Science',
  'website': 'https://www.britannica.com/science/computer-science',
  'content': 'Our editors will review what you’ve submitted and determine whether to revise the article.Computer science,  the study of computers and computing, including their theoretical and algorithmic foundations, hardware and software, and their uses for processing information. The discipline of computer science includes the study of algorithms and data structures, computer and network design, modeling data and information processes, and artificial intelligence. Computer science draws some of its foundations from mathematics and engineering and therefore incorporates techniques from areas such as queueing theory, probability and statistics, and electronic circuit design. Computer science also makes heavy use of hypothesis testing and experimentation during the conceptualization, design, measurement, and refinement of new algorithms, information structures, and co

In [2]:
# Gather the set of all unique terms

unique_terms = {term for doc in docs for term in doc["content"].split()}
unique_terms

{'waterfalls,',
 'heavy',
 'Council.[5]',
 'Mains',
 '2001),',
 'both',
 'industrialists,',
 'by,',
 'Report',
 'network',
 'mentor',
 'establishment',
 'Higher',
 'Applications',
 'here',
 'universities.Nationally',
 'Bilaspur,',
 'national',
 'exam',
 'Pop.',
 'abstractly,',
 'RACHI',
 'Via',
 'systems,',
 'Science,',
 'people',
 'lays',
 'municipality',
 'thinking,',
 'Enjoy',
 'Prelims',
 'years.The',
 'self-reliance',
 'Banaras',
 'SearchPlease',
 'University',
 'Telangana',
 'approximately',
 'Committee',
 'least',
 'focus.',
 'used',
 'computational',
 'Nagpur,',
 'Calcutta',
 'they',
 'Singh',
 'Pradesh',
 'campaign',
 'checking',
 'clear',
 '16th',
 '22',
 'increased',
 'Formal',
 'gained',
 '(MIT).',
 'Sheet',
 'IIT-BHU',
 'artificial',
 'Ranked',
 'Southern',
 'finds',
 'Ranking.',
 'working',
 'experimentation',
 'establishment,',
 'present,',
 'longer',
 'methodically,',
 'City/Town.',
 'incorporates',
 'doctoral',
 'modern',
 'into',
 'framework',
 'earning',
 'Top',
 'pa

In [3]:
# function for primary and secondary parameter for document ids

def para(ids):
    doc_no, query_no = map(lambda ids: int(ids[1:]), ids.split('_'))
    return (doc_no, query_no)

In [4]:
# Construct an inverted index
# here as a Python dictionary for ease of interpretability

# word stemming, stop word removing with nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  
from nltk.stem import PorterStemmer
stop_words = set(stopwords.words("english"))

ps = PorterStemmer()

inverted_index = {}
for doc in docs:
    for term in word_tokenize(doc["content"]):
        if term not in stop_words:
            term = ps.stem(term)
            if term in inverted_index:
                if para(doc["doc_id"]) not in inverted_index[term]:
                    inverted_index[term].append(para(doc["doc_id"]))
                    inverted_index[term][0] += 1
            else: inverted_index[term] = [1,para(doc["doc_id"])]

inverted_index

{'our': [3, (1, 1), (8, 3), (9, 3)],
 'editor': [3, (1, 1), (8, 3), (9, 3)],
 'review': [4, (1, 1), (3, 2), (8, 3), (9, 3)],
 '’': [10,
  (1, 1),
  (2, 1),
  (6, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (16, 2),
  (8, 3),
  (9, 3)],
 'submit': [3, (1, 1), (8, 3), (9, 3)],
 'determin': [3, (1, 1), (8, 3), (9, 3)],
 'whether': [3, (1, 1), (8, 3), (9, 3)],
 'revis': [3, (1, 1), (8, 3), (9, 3)],
 'article.comput': [1, (1, 1)],
 'scienc': [15,
  (1, 1),
  (2, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (9, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (7, 2),
  (9, 2),
  (11, 2)],
 ',': [34,
  (1, 1),
  (2, 1),
  (3, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (11, 1),
  (12, 1),
  (15, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (1, 2),
  (3, 2),
  (4, 2),
  (8, 2),
  (9, 2),
  (11, 2),
  (16, 2),
  (19, 2),
  (1, 3),
  (4, 3),
  (6, 3),
  (7, 3),
  (8, 3),
  (9, 3),
  (13, 3),
  (14, 3),
  (15, 3),
  (17, 3),
  (19, 3),
  (20, 3)],
 'studi': [7, (1, 1), 

In [5]:
# saving the file containing the document ids and corresponding contents (along with the webpage link)
import json

with open("./docs.json", "w") as f:
    f.write(json.dumps(docs, indent=4))

In [6]:
# word with thier frequency in documents

freq = {}
for word in inverted_index:
    if inverted_index[word][0] in freq:
        freq[inverted_index[word][0]].append(word)
    else:
        freq[inverted_index[word][0]] = [word]
freq

{3: ['our',
  'editor',
  'submit',
  'determin',
  'whether',
  'revis',
  'includ',
  'softwar',
  'inform',
  'design',
  'test',
  'world',
  '?',
  'power',
  'fundament',
  'peopl',
  'societi',
  'advanc',
  'work',
  'other',
  'candid',
  '2020',
  'covid-19',
  '2021',
  'video',
  'A',
  'innov',
  'support',
  'contribut',
  "'s",
  'secur',
  'need',
  'along',
  'provid',
  'career',
  'websit',
  '6',
  'these',
  'recogn',
  '1',
  'govern',
  'act',
  'exam',
  'kharagpur',
  'mumbai',
  'delhi',
  'issu',
  'you',
  'get',
  'form',
  '1945',
  'best',
  'top',
  'countri',
  'bombay',
  'sinc',
  'clear',
  'known',
  'intern',
  'toward',
  'west',
  'committe',
  'least',
  'popul',
  'time',
  'build',
  'ensur',
  'lakh',
  'south',
  'central',
  'madhya',
  'pradesh',
  'villag'],
 4: ['review',
  'algorithm',
  'use',
  'area',
  'theori',
  'make',
  'new',
  'stand',
  'redirect',
  'field',
  'help',
  'school',
  'creat',
  'link',
  'list',
  'number',
  

In [7]:
available_freq = sorted(freq)
available_freq

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 34, 38]

In [8]:
mostly_used = []
count = 3
for i in range(len(available_freq)-1,-1,-1):
    for item in freq[available_freq[i]]:
        if count==0:
            break
        mostly_used.append(item)
        count-=1
    if count==0:
        break

mostly_used

['.', ',', 'scienc']

In [9]:
least_used = []
count = 3
for i in range(0,len(available_freq)):
    for item in freq[available_freq[i]]:
        if count==0:
            break
        least_used.append(item)
        count-=1
    if count==0:
        break

least_used

['article.comput', 'theoret', 'disciplin']

# Part 2 : Merge posting-lists

Sort the posting list with their document ids

In [10]:
for terms in inverted_index:
    inverted_index[terms] = [inverted_index[terms][0]] + sorted(inverted_index[terms][1:])

inverted_index

{'our': [3, (1, 1), (8, 3), (9, 3)],
 'editor': [3, (1, 1), (8, 3), (9, 3)],
 'review': [4, (1, 1), (3, 2), (8, 3), (9, 3)],
 '’': [10,
  (1, 1),
  (2, 1),
  (6, 1),
  (8, 3),
  (9, 3),
  (15, 1),
  (16, 1),
  (16, 2),
  (17, 1),
  (18, 1)],
 'submit': [3, (1, 1), (8, 3), (9, 3)],
 'determin': [3, (1, 1), (8, 3), (9, 3)],
 'whether': [3, (1, 1), (8, 3), (9, 3)],
 'revis': [3, (1, 1), (8, 3), (9, 3)],
 'article.comput': [1, (1, 1)],
 'scienc': [15,
  (1, 1),
  (2, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (7, 2),
  (9, 1),
  (9, 2),
  (11, 2),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1)],
 ',': [34,
  (1, 1),
  (1, 2),
  (1, 3),
  (2, 1),
  (3, 1),
  (3, 2),
  (4, 2),
  (4, 3),
  (5, 1),
  (6, 1),
  (6, 3),
  (7, 1),
  (7, 3),
  (8, 1),
  (8, 2),
  (8, 3),
  (9, 2),
  (9, 3),
  (11, 1),
  (11, 2),
  (12, 1),
  (13, 3),
  (14, 3),
  (15, 1),
  (15, 3),
  (16, 2),
  (17, 1),
  (17, 3),
  (18, 1),
  (19, 1),
  (19, 2),
  (19, 3),
  (20, 1),
  (20, 3)],
 'studi': [7, (1, 1), 

Function for interesection in of two posting list

In [100]:
def Intersection(list1, list2):
    m,n = list1[0],list2[0]
    i,j = 1,1
    out = [0]
    while i <= m and j <= n:
        value1 = list1[i][1] + 3*list1[i][0]
        value2 = list2[j][1] + 3*list2[j][0]
        if value1 < value2: 
            i += 1
        elif value2 < value1: 
            j+= 1
        else: 
            out.append(list2[j])
            out[0]+=1
            j += 1
            i += 1
    return out

Function for merge posting list

In [101]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

def mergeList(query, IntersectionMethod=Intersection):
    '''
        Right now we are considering that user will give only one opration in the query that is AND as the
        question requirement
    '''
    terms_in_the_query = query.split()
    terms_in_the_query = [ps.stem(terms_in_the_query[i]) for i in range(len(terms_in_the_query)) if i%2==0]
    
    # sorting terms with posting list length 
    # it will optimize the performance if we firstly merge two list that are shortest
    terms_in_the_query.sort(key= lambda x: inverted_index[x][0])
    
    # Now we can merge the list until oe list remain
    
    out = IntersectionMethod(inverted_index[terms_in_the_query[0]], inverted_index[terms_in_the_query[1]])
    
    for i in range(2,len(terms_in_the_query)):
        out = IntersectionMethod(out, inverted_index[terms_in_the_query[i]])
    return out

In [102]:
# Running queries as given in the question
queries = ['IIT AND JEE', 'Raipur AND Bhilai', 'Raipur AND Bhilai AND Chhattisgarh']
results = {q: mergeList(q) for q in queries}

results

{'IIT AND JEE': [2, (8, 2), (16, 2)],
 'Raipur AND Bhilai': [1, (17, 3)],
 'Raipur AND Bhilai AND Chhattisgarh': [1, (17, 3)]}

In [103]:
# storing result with their document id in the json
for q in results:
    results[q] = [f'd{id[0]}_q{id[1]}' for id in results[q][1:]]

import json

with open("./part2_queries_results.json", "w") as f:
    f.write(json.dumps(results, indent=4))


# Part 3 : Adding Skip-pointers

In [104]:
# Function to create skip pointer list

import math
def getSkiplist(length):
    return [i for i in range(1, length, math.ceil(math.sqrt(length)))] + [-1]

getSkiplist(64)

[1, 9, 17, 25, 33, 41, 49, 57, -1]

In [130]:
# Function for the skip pointer intersection

import time

def skiplist_intersection(postingList1, postingList2):

    #initilization
    length1, length2 = postingList1[0],postingList2[0]
    skiplist1, skiplist2=getSkiplist(length1), getSkiplist(length2)
    normalPointer1, normalPointer2 = 1,1
    skipPointer1, skipPointer2=0,0
    result=[0]
    it=0
    
    #iterations
    while normalPointer1 <= length1 and normalPointer2 <= length2:
        value1, value2 = postingList1[normalPointer1][1] + 3*postingList1[normalPointer1][0], postingList2[normalPointer2][1] + 3*postingList2[normalPointer2][0]
        if value1 == value2:
            result.append(postingList1[normalPointer1])
            result[0]+=1
            normalPointer1+=1
            normalPointer2+=1
           
        elif value1 < value2:
            # first list on it's skip pointer if we it matches then we can have two options 
            if skiplist1[skipPointer1]==normalPointer1:
                nextindex=skiplist1[skipPointer1+1]
                while nextindex!=-1 and (postingList1[nextindex][1] + 3*postingList1[nextindex][0]) < value2:
                    normalPointer1 = nextindex
                    skipPointer1+=1
                    nextindex=skiplist1[skipPointer1+1]
            normalPointer1 += 1

        else:
            if skiplist2[skipPointer2]==normalPointer2:
                nextindex=skiplist2[skipPointer2+1]
                while nextindex!=-1 and (postingList2[nextindex][1] + 3*postingList2[nextindex][0]) < value1:
                    normalPointer2=nextindex
                    skipPointer2+=1
                    nextindex=skiplist2[skipPointer2+1]  
            normalPointer2 += 1
        
        #updating skip pointer    
        while  skiplist1[skipPointer1]!=-1 and normalPointer1 > skiplist1[skipPointer1]:
            skipPointer1+=1
        
        while skiplist2[skipPointer2]!=-1 and normalPointer2 > skiplist2[skipPointer2]:
            skipPointer2+=1
        
    return result



In [131]:
# Running queries as given in the question
queries = ['IIT AND JEE', 'Raipur AND Bhilai', 'Raipur AND Bhilai AND Chhattisgarh']
results = {q: mergeList(q, skiplist_intersection) for q in queries}

results

{'IIT AND JEE': [2, (8, 2), (16, 2)],
 'Raipur AND Bhilai': [1, (17, 3)],
 'Raipur AND Bhilai AND Chhattisgarh': [1, (17, 3)]}

### Comparison between skip pointer list intersection and normal list intersection

In [143]:
# calculate time in normal list intersection
start = time.time()
queries = ['IIT AND JEE', 'Raipur AND Bhilai', 'Raipur AND Bhilai AND Chhattisgarh']
results = {q: mergeList(q) for q in queries}
end = time.time()
time_in_normal_list_intersection = end-start
time_in_normal_list_intersection

0.001912832260131836

In [142]:
# calculate time in skip list intersection
start = time.time()
queries = ['IIT AND JEE', 'Raipur AND Bhilai', 'Raipur AND Bhilai AND Chhattisgarh']
results = {q: mergeList(q, skiplist_intersection) for q in queries}
end = time.time()
time_in_skip_list_intersection = end-start
time_in_skip_list_intersection

0.0008051395416259766

**This studies shows that skip list will reduce time, sometimes I am seeing I am getting more time because skip list more comparisons then normal list and list is very small here as atmost 60 document in each inverted index.**

# Part 4: Scoring

In [8]:
from nltk.tokenize import word_tokenize

tokenized_list = []

for doc in docs:
    tokenized_list.append({
        "doc_id" : doc["doc_id"],
        "tokens" : word_tokenize(doc["content"])
    })

In [9]:
#Vectorizing the documents

from py4tfidf.vectorizer import Tfidf
vec = Tfidf()

for i in range(len(tokenized_list)):
    tokenized_list[i]["vector"] = vec.vectorize_train(tokenized_list[i]["tokens"])


In [10]:
tokenized_list[0]


{'doc_id': 'd1_q1',
 'tokens': ['Our',
  'editors',
  'will',
  'review',
  'what',
  'you',
  '’',
  've',
  'submitted',
  'and',
  'determine',
  'whether',
  'to',
  'revise',
  'the',
  'article.Computer',
  'science',
  ',',
  'the',
  'study',
  'of',
  'computers',
  'and',
  'computing',
  ',',
  'including',
  'their',
  'theoretical',
  'and',
  'algorithmic',
  'foundations',
  ',',
  'hardware',
  'and',
  'software',
  ',',
  'and',
  'their',
  'uses',
  'for',
  'processing',
  'information',
  '.',
  'The',
  'discipline',
  'of',
  'computer',
  'science',
  'includes',
  'the',
  'study',
  'of',
  'algorithms',
  'and',
  'data',
  'structures',
  ',',
  'computer',
  'and',
  'network',
  'design',
  ',',
  'modeling',
  'data',
  'and',
  'information',
  'processes',
  ',',
  'and',
  'artificial',
  'intelligence',
  '.',
  'Computer',
  'science',
  'draws',
  'some',
  'of',
  'its',
  'foundations',
  'from',
  'mathematics',
  'and',
  'engineering',
  'and'